This is a classification example to show how to use Oboe for training and testing, in the context of AutoML.

In [1]:
# necessary modules
import sys
import pandas as pd
import os
import time
import numpy as np
import multiprocessing

#Oboe modules; this will be simplified when Oboe becomes pip installable
automl_path = '../automl/'
sys.path.append(automl_path)
from auto_learner import AutoLearner
import util

#import scikit-learn modules
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
#load and split dataset into training and test folds
data = load_iris()
x = np.array(data['data'])
y = np.array(data['target'])
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

# Example 1: a no-brainer use

In [3]:
# # initialize the autolearner class
# m = AutoLearner(p_type='classification', runtime_limit=10)

In [4]:
# # fit autolearner on training set and record runtime
# start = time.time()
# m.fit(x_train, y_train)
# elapsed_time = time.time() - start

In [5]:
# # use the fitted autolearner for prediction on test set
# y_predicted = m.predict(x_test)
# print("prediction error: {}".format(util.error(y_test, y_predicted, 'classification')))    
# print("elapsed time: {}".format(elapsed_time))

In [6]:
# # get names of the selected machine learning models
# m.get_models()

# Example 2: build an ensemble of models

In [7]:
#experimental settings
VERBOSE = False #whether to print out information indicating current fitting progress
N_CORES = 1 #number of cores
RUNTIME_BUDGET = 512

In [8]:
#optional: limit the types of algorithms
s = ['AB', 'ExtraTrees', 'GNB', 'KNN', 'RF', 'DT']

In [9]:
#autolearner arguments
autolearner_kwargs = {
    'p_type': 'classification',
    'runtime_limit': RUNTIME_BUDGET,
    'verbose': VERBOSE,
    'selection_method': 'min_variance',
    'algorithms': s,
    'stacking_alg': 'greedy',
    'n_cores': N_CORES,
    'build_ensemble': True,
    'coca': True,
}

In [10]:
#intialize the autolearner class
m = AutoLearner(**autolearner_kwargs)

In [11]:
# fit autolearner on training set and record runtime
start = time.time()
m.fit(x_train, y_train)
elapsed_time = time.time() - start

../automl/convex_opt.py:51: RuntimeWarning: invalid value encountered in double_scalars
  return -1 * sign * log_det
/anaconda3/lib/python3.6/site-packages/numpy/linalg/linalg.py:1817: RuntimeWarning: invalid value encountered in slogdet
  sign, logdet = _umath_linalg.slogdet(a, signature=signature)
../automl/auto_learner.py:136: RuntimeWarning: invalid value encountered in greater
  to_sample = valid[np.where(v_opt > 0.9)[0]]
../automl/convex_opt.py:51: RuntimeWarning: invalid value encountered in double_scalars
  return -1 * sign * log_det
/anaconda3/lib/python3.6/site-packages/numpy/linalg/linalg.py:1817: RuntimeWarning: invalid value encountered in slogdet
  sign, logdet = _umath_linalg.slogdet(a, signature=signature)
../automl/auto_learner.py:136: RuntimeWarning: invalid value encountered in greater
  to_sample = valid[np.where(v_opt > 0.9)[0]]


In [12]:
# use the fitted autolearner for prediction on test set
y_predicted = m.predict(x_test)
print("prediction error: {}".format(util.error(y_test, y_predicted, 'classification')))
print("elapsed time: {}".format(elapsed_time))
print("individual accuracies of selected models: {}".format(m.get_model_accuracy(y_test)))

prediction error: 0.03670634920634921
elapsed time: 344.23552417755127
individual accuracies of selected models: [0.03670634920634921, 0.03670634920634921, 0.03670634920634921]


In [13]:
# get names of the selected machine learning models
m.get_models()

{'ensemble method': 'greedy selection',
 'base learners': {'AB': [{'n_estimators': 50, 'learning_rate': 2.5},
   {'n_estimators': 50, 'learning_rate': 2.5},
   {'n_estimators': 50, 'learning_rate': 1}]}}

# Example 3: just select a collection of promising models without building an ensemble afterwards

In [14]:
#experimental settings
VERBOSE = False #whether to print out information indicating current fitting progress
N_CORES = 1 #number of cores
RUNTIME_BUDGET = 15

In [15]:
#optional: limit the types of algorithms
s = ['AB', 'ExtraTrees', 'GNB', 'KNN', 'RF', 'DT']

In [16]:
#autolearner arguments
autolearner_kwargs = {
    'p_type': 'classification',
    'runtime_limit': RUNTIME_BUDGET,
    'verbose': VERBOSE,
    'selection_method': 'min_variance',
    'algorithms': s,
    'stacking_alg': 'greedy',
    'n_cores': N_CORES,
    'build_ensemble': False,
}

In [17]:
#intialize the autolearner class
m = AutoLearner(**autolearner_kwargs)

In [18]:
# fit autolearner on training set and record runtime
start = time.time()
m.fit(x_train, y_train)
elapsed_time = time.time() - start

../automl/convex_opt.py:51: RuntimeWarning: invalid value encountered in double_scalars
  return -1 * sign * log_det
/anaconda3/lib/python3.6/site-packages/numpy/linalg/linalg.py:1817: RuntimeWarning: invalid value encountered in slogdet
  sign, logdet = _umath_linalg.slogdet(a, signature=signature)
../automl/auto_learner.py:136: RuntimeWarning: invalid value encountered in greater
  to_sample = valid[np.where(v_opt > 0.9)[0]]
../automl/convex_opt.py:51: RuntimeWarning: invalid value encountered in double_scalars
  return -1 * sign * log_det
/anaconda3/lib/python3.6/site-packages/numpy/linalg/linalg.py:1817: RuntimeWarning: invalid value encountered in slogdet
  sign, logdet = _umath_linalg.slogdet(a, signature=signature)
../automl/auto_learner.py:136: RuntimeWarning: invalid value encountered in greater
  to_sample = valid[np.where(v_opt > 0.9)[0]]
../automl/convex_opt.py:51: RuntimeWarning: invalid value encountered in double_scalars
  return -1 * sign * log_det
/anaconda3/lib/python

In [19]:
# use the fitted autolearner for prediction on test set
y_predicted = m.predict(x_test)
 
print("elapsed time: {}".format(elapsed_time))
print("accuracies of selected models: {}".format(m.get_model_accuracy(y_test)))

elapsed time: 7.7686731815338135
accuracies of selected models: [0.01835317460317461, 0.01835317460317461, 0.01835317460317461, 0.03670634920634921, 0.03670634920634921, 0.03670634920634921, 0.03670634920634921, 0.03670634920634921, 0.03670634920634921, 0.03670634920634921, 0.03670634920634921, 0.03670634920634921, 0.01835317460317461, 0.03670634920634921, 0.03670634920634921, 0.03670634920634921, 0.03670634920634921, 0.01835317460317461, 0.03670634920634921, 0.03670634920634921, 0.03670634920634921, 0.03670634920634921, 0.03670634920634921, 0.03670634920634921, 0.03670634920634921, 0.03670634920634921, 0.03670634920634921, 0.03670634920634921, 0.03670634920634921, 0.03670634920634921, 0.03670634920634921, 0.03670634920634921, 0.03670634920634921, 0.03670634920634921, 0.03670634920634921, 0.03670634920634921, 0.03670634920634921, 0.055059523809523815, 0.03670634920634921, 0.03670634920634921, 0.03670634920634921, 0.03670634920634921, 0.03670634920634921, 0.055059523809523815, 0.0367063

Note that we do not have a single accuracy value here if we do not build an ensemble, instead, we just have a collection of fitted models with individual accuracies reported.

In [20]:
# get names of the selected machine learning models
m.get_models()

{'KNN': [{'n_neighbors': 1, 'p': 1},
  {'n_neighbors': 1, 'p': 1},
  {'n_neighbors': 1, 'p': 2},
  {'n_neighbors': 3, 'p': 1},
  {'n_neighbors': 3, 'p': 2},
  {'n_neighbors': 5, 'p': 1},
  {'n_neighbors': 5, 'p': 2},
  {'n_neighbors': 7, 'p': 1},
  {'n_neighbors': 7, 'p': 2},
  {'n_neighbors': 9, 'p': 1},
  {'n_neighbors': 9, 'p': 2},
  {'n_neighbors': 11, 'p': 1},
  {'n_neighbors': 13, 'p': 1},
  {'n_neighbors': 11, 'p': 2},
  {'n_neighbors': 15, 'p': 1},
  {'n_neighbors': 13, 'p': 2},
  {'n_neighbors': 15, 'p': 2}],
 'ExtraTrees': [{'min_samples_split': 0.0001, 'criterion': 'gini'},
  {'min_samples_split': 1e-05, 'criterion': 'gini'},
  {'min_samples_split': 2, 'criterion': 'gini'},
  {'min_samples_split': 0.0001, 'criterion': 'entropy'},
  {'min_samples_split': 1e-05, 'criterion': 'entropy'},
  {'min_samples_split': 2, 'criterion': 'entropy'},
  {'min_samples_split': 4, 'criterion': 'entropy'},
  {'min_samples_split': 0.001, 'criterion': 'gini'},
  {'min_samples_split': 0.001, 'crit